# Data Cleansing (Limpeza de dados)
Para a etapa de data cleansing, serão utilizados os seguintes pacotes: pandas (estruturas de Series e DataFrame), numpy (para tratamento de valores NaN), requests (para efetuação de requisições e processamento das responses) e json (também processamento das responses).

In [4]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
games_df = pd.read_csv('datasets/metacritic_games.csv')

In [5]:
pd.set_option('display.max_columns', len(games_df))
pd.set_option('display.max_rows', 100)

### Verificando quais colunas possuem valores nulos ou missing
Podemos verificar que as colunas developer, genre, number_players e rating possuem valores faltantes. Colunas como gênero e número de jogadores, para as análises que iremos realizar, não serão relevantes, então tudo bem apresentarem valores missing. Porém, queremos os valores para desenvolvedora e avaliação.

In [6]:
games_df.isna().any()

game                False
platform            False
developer            True
genre                True
number_players       True
rating               True
release_date        False
positive_critics    False
neutral_critics     False
negative_critics    False
positive_users      False
neutral_users       False
negative_users      False
metascore           False
user_score          False
dtype: bool

### Manipulação dos valores missing
Como citado anteriormente, iremos nos preocupar com valores missing presentes nas colunas developer e rating. Por isso, iremos preencher os valores missing com dados obtidos de uma API com informações públicas de videogames chamada IGDB. Devido a estrutura da API, serão criadas três funções: a primeira para obter o id das companhias envolvidas no desenvolvimento a partir do nome do jogo passado como param, a segunda para obter o id da companhia principal envolvida a partir do id das companhias e a terceira para obter o nome propriamente da companhia principal. 

É importante destacar que a informação acerca da desenvolvedora de quatro jogos não constava nem mesmo na API, então foram preenchidas manualmente.

In [7]:
def requestInvolvedCompanies(game):
    url = "https://api-v3.igdb.com/games"
    
    payload = "search \"{}\";\nfields involved_companies;\nrelease_date.human;\nwhere version_parent = null;".format(game)
    
    headers = {
    'cookie': "__cfduid=d4bed4e84314697f5dcf282a31e3174bd1593302851",
    'user-key': "2078e56b79acab9b669a3bd18661c2ba"
    }
    
    try:
        response = requests.request("GET", url, data=payload, headers=headers)
    
        involved_companies = json.loads(response.text)    
        return involved_companies[0]['involved_companies'][0]
    
    except:
        print('Erro durante o jogo {}'.format(game))

In [8]:
def requestMainCompany(id_company):
    url = "https://api-v3.igdb.com/involved_companies"
    
    payload = "fields company; where id = {};\n".format(id_company)
    
    headers = {
    'cookie': "__cfduid=d4bed4e84314697f5dcf282a31e3174bd1593302851",
    'user-key': "2078e56b79acab9b669a3bd18661c2ba"
    }
    
    try:
        response = requests.request("GET", url, data=payload, headers=headers)        
        company = json.loads(response.text)
    
        return company[0]['company']
       
    except:
        print('Erro durante o id_company {}'.format(id_company))

In [9]:
def requestDeveloperName(id_developer):
    url = "https://api-v3.igdb.com/companies"
    
    payload = "fields *;\nwhere id = {};".format(id_developer)
    headers = {
    'cookie': "__cfduid=d4bed4e84314697f5dcf282a31e3174bd1593302851",
    'user-key': "2078e56b79acab9b669a3bd18661c2ba"
    }
    
    try:
        response = requests.request("GET", url, data=payload, headers=headers)        
        developer = json.loads(response.text)[0]
        return developer['name']
    
    except:
        print('Erro durante o id_developer {}'.format(id_developer))

Podemos verificar as informações de jogos que possuem o valor vazio na coluna developer

In [10]:
games_df[games_df['developer'].isna()]

,game,platform,developer,genre,number_players,rating,release_date,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,metascore,user_score
258,"Harry Potter and the Deathly Hallows, Part 2",PC,NaN,Action,NaN,T,"Jul 12, 2011",1,1,10,8,0,8,43,46
620,Cannon Fodder 3,PC,NaN,Strategy,1 Player,NaN,"Feb 9, 2012",1,6,3,0,1,1,49,57
1103,Seduce Me,PC,NaN,Strategy,1 Player,AO,"Jan 2, 2013",0,5,7,2,0,4,41,34
1434,Outlast: Whistleblower,PC,NaN,Action Adventure,NaN,M,"May 6, 2014",6,6,0,20,5,3,73,79
1562,Dead Nation,VITA,NaN,Action,NaN,M,"Apr 15, 2014",6,8,0,4,1,4,68,79
1609,Memento Mori 2,PC,NaN,NaN,NaN,NaN,"May 13, 2014",0,0,0,0,0,0,65,75
1818,Destiny: The Taken King,PS4,NaN,NaN,NaN,NaN,"Sep 15, 2015",0,0,0,0,0,0,86,62
3500,NightCry,PC,NaN,NaN,NaN,NaN,"Mar 28, 2016",0,0,0,0,0,0,46,30
5520,Wild West Online,PC,NaN,Role-Playing,Online Multiplayer,NaN,"May 10, 2018",0,0,8,0,0,8,29,1


Para mais tarde processarmos os valores missing da coluna developer, será criada uma Series que irá conter todos os jogos que não possuem uma desenvolvedora atrelada. 

In [11]:
games = games_df[games_df['developer'].isna()]['game']
games

258     Harry Potter and the Deathly Hallows, Part 2
620                                  Cannon Fodder 3
1103                                       Seduce Me
1434                          Outlast: Whistleblower
1562                                     Dead Nation
1609                                  Memento Mori 2
1818                         Destiny: The Taken King
3500                                        NightCry
5520                                Wild West Online
Name: game, dtype: object

Com nossas funções de acesso aos dados da API, podemos então percorrer os dados de nome do jogo do nosso dataframe e substituir os valores missing em nome da desenvolvedora

In [12]:
devs = list()

for game in games:
    id_company = requestInvolvedCompanies(game)
    id_developer = requestMainCompany(id_company)
    developer_name = requestDeveloperName(id_developer)
    devs.append(developer_name)
    
print(devs)

['Electronic Arts',
 'Burut CT',
 'No Reply Games',
 'Red Barrels',
 'Housemarque',
 'Centauri Production',
 'Bungie',
 'Nude Maker',
 '612 Games']

Com os nomes das desenvolvedoras que antes estavam ausentes em mãos, podemos então atribuir cada uma ao seu devido lugar (ou jogo). Para isso, usaremos uma estrutura numérica do pandas que irá servir de container para os índices de cada jogo que não possui uma desenvolvedora atrelada. 

In [13]:
index_missing = games_df[games_df['developer'].isna()].index
print(index_missing)

Int64Index([258, 620, 1103, 1434, 1562, 1609, 1818, 3500, 5520], dtype='int64')


Após isso, iremos percorrer a lista de índices, passando cada índice como parâmetro para o método `iloc[]` do DataFrame e como valor o índice correspondente na lista de developers.

In [15]:
iterator = 0
while iterator < len(index_missing):
    games_df['developer'].iloc[index_missing[iterator]] = devs[iterator]
    iterator +=1

/home/wabbajack/data-science/data-science-env/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Vamos mais uma vez verificar as informações de jogos que possuíam valores missing, em busca de eventuais remanescentes.

In [16]:
games_df[games_df['developer'].isna()]

,game,platform,developer,genre,number_players,rating,release_date,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,metascore,user_score


É possível confirmar que o DataFrame que deveria conter a relação de jogos que não possuem uma desenvolvedora está vazio, ou seja, o nome da desenvolvedora foi atribuído corretamente ao seu jogo correspondente! Cheers =)

In [18]:
games_df.to_csv('datasets/cleaned_metacritic.csv', sep=';')